In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle 

In [5]:
data=pd.read_csv("churn_modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#drp irreelevent features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

In [8]:
#one hot inco geogghraphy
from sklearn.preprocessing import OneHotEncoder
eco=OneHotEncoder()
geo_encoder=eco.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
eco.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
geo_encoderdf=pd.DataFrame(geo_encoder.toarray(),columns=eco.get_feature_names_out(['Geography']))

In [12]:
print(pd.DataFrame)


<class 'pandas.core.frame.DataFrame'>


In [13]:
geo_encoderdf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
#combine 
data=pd.concat([data.drop('Geography',axis=1),geo_encoderdf],axis=1)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
### save the encoder
with open ('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)
with open('eco.pkl','wb')as file:
    pickle.dump(eco,file)  
  

In [17]:
#divide into depend and not depend smjh ni aya
x=data.drop('Exited',axis=1)
y=data['Exited']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [18]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [19]:
with open ('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [21]:
#ann model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_test.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
#compilation
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [24]:
#tensor board  setup
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [25]:
#early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [26]:
#training
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    
)

Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 0.4395 - accuracy: 0.8117 - val_loss: 0.3767 - val_accuracy: 0.8445
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3713 - accuracy: 0.8470 - val_loss: 0.3545 - val_accuracy: 0.8520
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3529 - accuracy: 0.8533 - val_loss: 0.3473 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3437 - accuracy: 0.8586 - val_loss: 0.3448 - val_accuracy: 0.8570
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3397 - accuracy: 0.8585 - val_loss: 0.3451 - val_accuracy: 0.8580
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3373 - accuracy: 0.8614 - val_loss: 0.3432 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3339 - accuracy: 0.8641 - val_loss: 0.3378 - val_accuracy: 0.86

In [27]:
model.save('model.h5')

e:\ann classi project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
#load tensorboard extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16972), started 1:34:36 ago. (Use '!kill 16972' to kill it.)

In [30]:
#load the pickle file
